In [1]:
import pandas as pd
import numpy as np
import csv

import spacy
spacy.prefer_gpu()

import pickle
import sqlite3
import re

Do data manipulation in SQL instead.

In [ ]:
try:
    # data of both wikipedia pages 
    sample = pd.read_csv('../data/plaintext_link_sample_full.csv')
    
except FileNotFoundError:
    
    # connect to database
    db = sqlite3.connect('../data/kensho.db')
    c = db.cursor()

    # visualize query into pandas dataframe
    def viz_tables(cols, query):
        q = c.execute(query).fetchall()
        framelist = dict()
        for i, col_name in enumerate(cols):
            framelist[col_name] = [col[i] for col in q]
        return pd.DataFrame.from_dict(framelist)
    
    # column names
    plaintext_link_cols = [col[1] for col in c.execute("PRAGMA table_info(plaintext_link)")]
    
    # get a sample of the data to work on pandas
    query2 = """
    SELECT *
    FROM plaintext_link
    """
    sample = viz_tables(plaintext_link_cols, query2)
    sample.to_csv('../data/plaintext_link_sample_full.csv', index=False)
    
    c.close()
    db.close()

In [ ]:
sample.head()

***

In [3]:
from text_cleaning_functions import *

unicode_dict = get_unicode_dict()

# replace html encoded strings, weird characters, and additional whitespaces
cleaned_sample = text_preprocessing(sample, [('&\w+;|&#[0-9]+;|&#[xX][a-fA-F0-9]+;', ''), 
                                             ('[^a-zA-Z0-9\s]', ''), 
                                             ('\s{2,}', ' '),
                                             ('^ | $', '')], unicode_dict)

# approx 10-15 minutes for 5000 text...
true_entity, ner_entity = extract_entities_and_labels(cleaned_sample)

# keep entities which were both identified by NER and has a corresponding Wikipedia link
ner_entity, true_entity = drop_entities(ner_entity, true_entity)

In [4]:
ner_entity[0]

['an abugida from geez abugida or alphasyllabary is a segmental in which consonantvowel sequences are written as a unit each unit is based on a consonant letter and vowel notation is secondary this contrasts with a full alphabet in which vowels have status equal to consonants and with an abjad in which vowel marking is absent or optional although in less formal contexts all three types of script may be termed alphabets the terms also contrast them with a syllabary in which the symbols cannot be split into separate consonants and vowels abugidas include the extensive brahmic family of scripts of south and southeast asia semitic ethiopic scripts and canadian aboriginal syllabics which are themselves based in part on brahmic scripts as is the case for syllabaries the units of the writing system may consist of the representations both of syllables and of consonants for scripts of the brahmic family the term akshara is used for the units abugida as a term in linguistics was proposed by pete